In [4]:
%reload_ext autoreload
%autoreload 2

In [7]:
from scripts.agents import build_openai_agent, build_hf_agent
from scripts.run_agents import answer_questions
from scripts.optimize_prompt import optimize_prompt

In [8]:
import datasets

math_dataset = (
    datasets.load_dataset("reasoning-machines/gsm-hard", "main")["train"]
    .shuffle(seed=42)
    .select(range(30))
)
math_dataset = math_dataset.rename_column("input", "question")
math_dataset = math_dataset.rename_column("target", "answer")

task_column = ["GSM8K-hard"] * len(math_dataset)
math_dataset = math_dataset.add_column("task", task_column).select_columns(
    ["question", "answer", "task"]
)
eval_dataset = math_dataset

OUTPUT_DIR = "output_gsm8k-hard"

/home/ubuntu/venv/agents/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
